## Titanic competition Sikorskyi ##





**Downloading datasets from kaggle**

Using Google Colabs functions download Kaggle token. 


In [1]:
import pandas as pd
import numpy as np
import re
from google.colab import files
files.upload() #here you will download kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dsikora","key":"cbc2ceb0e7254cedca8d34555971dee5"}'}

Set permission to before downloading Titanic dataset.

In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json #set permission

kaggle.json


Downloading dataset.


In [3]:
!kaggle competitions download -c titanic 

  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 35.0MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 60.5MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.77MB/s]


Getting our data.


In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


**Data transformation**

Befor creating model, it is necessary to analyze, then create function to clean data.

Sibsp and Parch refer to family, so it is logical to replace them with one column and create 'Alone' column for people without family members.

In [5]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index = False).mean()

,FamilySize,Survived
0,1,0.303538
1,2,0.552795
2,3,0.578431
3,4,0.724138
4,5,0.200000
5,6,0.136364
6,7,0.333333
7,8,0.000000
8,11,0.000000


In [6]:
train['Alone'] = 0
train.loc[train['FamilySize'] == 1, 'Alone'] = 1
train[['Alone', 'Survived']].groupby(['Alone'], as_index=False).mean()

,Alone,Survived
0,0,0.505650
1,1,0.303538


'Fare' has NaNs, so they will be replaced to mean. Also, for future model it is better to replace 'Fare' with its categorical representation. 

In [7]:
train['Fare'] = train['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 5)
train[['CategoricalFare', 'Survived']].groupby(['CategoricalFare'], as_index=False).mean()

,CategoricalFare,Survived
0,"(-0.001, 7.854]",0.217877
1,"(7.854, 10.5]",0.201087
2,"(10.5, 21.679]",0.424419
3,"(21.679, 39.688]",0.444444
4,"(39.688, 512.329]",0.642045


Also, it is very useful to notice whether passanger had a cabin or not and which cabin.


In [8]:
def get_cabin(cabin):
  for i in str(cabin):
    if i in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Z']:
      return i
train['Cabin'] = train['Cabin'].fillna('Z')
train['BL'] = train['Cabin'].apply(get_cabin)
train[['BL', 'Survived']].groupby(['BL'], as_index=False).mean()

,BL,Survived
0,A,0.466667
1,B,0.744681
2,C,0.593220
3,D,0.757576
4,E,0.750000
5,F,0.615385
6,G,0.500000
7,Z,0.299854


In 'Age' we will replace NaNs with mean random and transform to categorical.

In [10]:
age_avg        = train['Age'].mean()
age_std        = train['Age'].std()    
age_null_count = train['Age'].isnull().sum()
    
age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size = age_null_count)
train['Age'][np.isnan(train['Age'])] = age_null_random_list
train['Age'] = train['Age'].astype(int)
    
train['CategoricalAge'] = pd.cut(train['Age'], 5)

train[['CategoricalAge', 'Survived']].groupby(['CategoricalAge'], as_index=False).mean()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CategoricalAge,Survived
0,"(-0.08, 16.0]",0.517241
1,"(16.0, 32.0]",0.346241
2,"(32.0, 48.0]",0.386719
3,"(48.0, 64.0]",0.434783
4,"(64.0, 80.0]",0.090909


We can find out different titles of passangers.

In [11]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

train['Title'] = train['Name'].apply(get_title)

pd.crosstab(train['Title'], train['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


So we can transform titles to categorical.

In [12]:
train['Title'] = train['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
    'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

train['Title'] = train['Title'].replace('Mlle', 'Miss')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')

train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


Column 'Embarked' has NaNs that will be replced with median.

In [13]:
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])

Adding Ageclass feature.


In [14]:
train['Age*Class'] = train.Age * train.Pclass 
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Alone,CategoricalFare,BL,CategoricalAge,Title,Age*Class
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,Z,S,2,0,"(-0.001, 7.854]",Z,"(16.0, 32.0]",Mr,66
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,2,0,"(39.688, 512.329]",C,"(32.0, 48.0]",Mrs,38
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,Z,S,1,1,"(7.854, 10.5]",Z,"(16.0, 32.0]",Miss,78
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,2,0,"(39.688, 512.329]",C,"(32.0, 48.0]",Mrs,35
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,Z,S,1,1,"(7.854, 10.5]",Z,"(32.0, 48.0]",Mr,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27,0,0,211536,13.0000,Z,S,1,1,"(10.5, 21.679]",Z,"(16.0, 32.0]",Rare,54
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.0000,B42,S,1,1,"(21.679, 39.688]",B,"(16.0, 32.0]",Miss,19
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,23,1,2,W./C. 6607,23.4500,Z,S,4,0,"(21.679, 39.688]",Z,"(16.0, 32.0]",Miss,69
889,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.0000,C148,C,1,1,"(21.679, 39.688]",C,"(16.0, 32.0]",Mr,26


Dropping useless columns.

In [15]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp',
                         'Parch', 'FamilySize']

train = train.drop(drop_elements, axis=1)
train

,Survived,Pclass,Sex,Age,Fare,Embarked,Alone,CategoricalFare,BL,CategoricalAge,Title,Age*Class
0,0,3,male,22,7.2500,S,0,"(-0.001, 7.854]",Z,"(16.0, 32.0]",Mr,66
1,1,1,female,38,71.2833,C,0,"(39.688, 512.329]",C,"(32.0, 48.0]",Mrs,38
2,1,3,female,26,7.9250,S,1,"(7.854, 10.5]",Z,"(16.0, 32.0]",Miss,78
3,1,1,female,35,53.1000,S,0,"(39.688, 512.329]",C,"(32.0, 48.0]",Mrs,35
4,0,3,male,35,8.0500,S,1,"(7.854, 10.5]",Z,"(32.0, 48.0]",Mr,105
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27,13.0000,S,1,"(10.5, 21.679]",Z,"(16.0, 32.0]",Rare,54
887,1,1,female,19,30.0000,S,1,"(21.679, 39.688]",B,"(16.0, 32.0]",Miss,19
888,0,3,female,23,23.4500,S,0,"(21.679, 39.688]",Z,"(16.0, 32.0]",Miss,69
889,1,1,male,26,30.0000,C,1,"(21.679, 39.688]",C,"(16.0, 32.0]",Mr,26


Creating function for data preprocessing and dividing into training and validation data.

In [16]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
def get_title(name):
        pattern = ' ([A-Za-z]+)\.'
        title_search = re.search(pattern, name)
        # If the title exists, extract and return it.
        if title_search:
            return title_search.group(1)
        return ""
def get_cabin(cabin):
  for i in str(cabin):
    if i in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Z']:
      return i
def load_data(name):
        dataset = pd.read_csv(name)
        # columns combination
        dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

        # replace value
        dataset['Alone'] = 0
        dataset.loc[dataset['FamilySize'] == 1, 'Alone'] = 1

        # fill Nan with mode
        dataset['Embarked'] = dataset['Embarked'].fillna(dataset['Embarked'].mode()[0])

        # fill Nan with median
        dataset['Fare'] = dataset['Fare'].fillna(dataset['Fare'].median())
        # binning with qcut
        dataset['Fare'] = pd.qcut(dataset['Fare'], 4)

        # fill Nan with mean
        age_avg = dataset['Age'].mean()
        age_std = dataset['Age'].std()
        age_null_count = dataset['Age'].isnull().sum()
        rng = np.random.RandomState(42)
        age_null_random_list = rng.uniform(age_avg - age_std, age_avg + age_std, size=age_null_count)
        dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list

        # binning with cut
        dataset['Age'] = pd.cut(dataset['Age'], 5)

        # apply regex
        dataset['Title'] = dataset['Name'].apply(get_title)
        # replace
        dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don',
                                                               'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],
                                                               'Rare')
        # replace
        dataset['Title'] = dataset['Title'].replace(['Mlle', 'Ms'], 'Miss')
        # replace
        dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
        # fill nans
        dataset['Title'] = dataset['Title'].fillna(0)
        #fill nans
        dataset['Cabin'] = dataset['Cabin'].fillna('Z')
        #apply replacement 
        dataset['BL'] = str(dataset['Cabin'].apply(get_cabin))
        


        # drop columns
        drop_elements = ['PassengerId', 'Name', 'Ticket', 'SibSp',
                         'Parch', 'FamilySize', 'Cabin']

        dataset = dataset.drop(drop_elements, axis=1)

        # encode labels
        le = LabelEncoder()

        le.fit(dataset['Sex'])
        dataset['Sex'] = le.transform(dataset['Sex'])
        
        le.fit(dataset['Title'])
        dataset['Title'] = le.transform(dataset['Title'])

        le.fit(dataset['Embarked'].values)
        dataset['Embarked'] = le.transform(dataset['Embarked'].values)

        le.fit(dataset['BL'])
        dataset['BL'] = le.transform(dataset['BL'])

        le.fit(dataset['Fare'])
        dataset['Fare'] = le.transform(dataset['Fare'])

        le.fit(dataset['Age'])
        dataset['Age'] = le.transform(dataset['Age'])
        #creating ageclass feature
        dataset['Age*Class'] = dataset.Age * dataset.Pclass
        le.fit(dataset['Age*Class'])
        dataset['Age*Class'] = le.transform(dataset['Age*Class'])

        return dataset
dataset = load_data("train.csv")
train, val = train_test_split(dataset, test_size=0.1)
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Alone', 'Title', 'BL', 'Age*Class']
training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(train[features].values, tf.float32),
            tf.cast(train['Survived'].values, tf.int32)
        )
    )
)
validation_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(val[features].values, tf.float32),
            tf.cast(val['Survived'].values, tf.int32)
        )
    )
)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


**Creating model**


In [17]:
import tensorflow as tf
import math
from tensorflow.keras.layers import Dense, Dropout
model = tf.keras.Sequential([
              Dense(12, activation = tf.nn.leaky_relu, input_shape = [9]), 
              Dropout(0.2),
              Dense(5, activation = tf.nn.leaky_relu),
              Dense(2, activation = tf.nn.softmax)
])
opt = tf.keras.optimizers.Adam(
    learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True,
    name='Adam'
)
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

num_train_examples = [i for i,_ in enumerate(training_dataset)][-1] + 1
num_val_examples = [i for i,_ in enumerate(validation_dataset)][-1] + 1
BATCH_SIZE = 256
train_dataset = training_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
val_dataset = validation_dataset.cache().batch(BATCH_SIZE)
model.fit(train_dataset, validation_data = val_dataset, epochs=1000, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Epoch 1/1000
4/4 [==============================] - 1s 117ms/step - loss: 0.7635 - accuracy: 0.4348 - val_loss: 0.6232 - val_accuracy: 0.6556
Epoch 2/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.6379 - accuracy: 0.6147 - val_loss: 0.5870 - val_accuracy: 0.6556
Epoch 3/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.6152 - accuracy: 0.6275 - val_loss: 0.5733 - val_accuracy: 0.6889
Epoch 4/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.5985 - accuracy: 0.6602 - val_loss: 0.5631 - val_accuracy: 0.7111
Epoch 5/1000
4/4 [==============================] - 0s 49ms/step - loss: 0.5772 - accuracy: 0.7059 - val_loss: 0.5664 - val_accuracy: 0.7111
Epoch 6/1000
4/4 [==============================] - 0s 13ms/step - loss: 0.6057 - accuracy: 0.6897 - val_loss: 0.5606 - val_accuracy: 0.7111
Epoch 7/1000
4/4 [==============================] - 0s 11ms/step - loss: 0.5841 - accuracy: 0.6965 - val_loss: 0.5414 - val_accuracy: 0.7111
Epoch 8/1000

Predicting and saving.

In [18]:
model.save('/content/model.h5')
test_ids = pd.read_csv("test.csv")["PassengerId"]
test_dataset = load_data("test.csv")
Y_pred = model.predict(test_dataset)
Y_pred = (Y_pred[:, 1]>=0.8).astype(int) 

submission = pd.DataFrame({
        "PassengerId": test_ids,
        "Survived": Y_pred
    })
submission.to_csv('/content/titanic.csv', index=False)
print('Done!')

Done!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
